# <center><span style="font-size: 42px;color: darkgreen;">Lab2: Clusterização</center></span>

<br>

---

<br>

## Contexto

Neste laboratório, exploraremos o conceito de **Clusterização**, uma técnica de **aprendizagem não supervisionada**. Foi fornecido um conjunto de dados composto por 7 arquivos `.txt`, cada um contendo uma notícia. O **objetivo** deste laboratório é utilizar algoritmos de clusterização para identificar automaticamente grupos de notícias semelhantes, sem conhecimento prévio de rótulos ou categorias específicas.

<br>

## Problema de Negócio

Dada a necessidade de organizar e categorizar grandes volumes de notícias, este laboratório busca **resolver o problema de agrupar notícias semelhantes em clusters**. Com essa abordagem, é possível identificar temas ou tópicos recorrentes, facilitando a análise e o entendimento dos conteúdos sem intervenção manual. A solução é especialmente útil para empresas de mídia, plataformas de notícias e qualquer organização que precise organizar informações textuais em larga escala de forma eficiente.


<br><br>

---

<br><br>

# <center>Iniciando o Laboratório</center>

<br><br>



## 1 Iniciando os Serviços

<br>

- **1.1 Iniciar o HDFS (NameNode, DataNode, SecondaryNameNode)**:
   ```bash
   start-dfs.sh  |  stop-dfs.sh
   ```
- **1.2 Iniciar o YARN (ResourceManager, NodeManager)**:
   ```bash
   start-yarn.sh  |  stop-yarn.sh
   ```
- **1.3 Verificando serviços**:
   ```bash
   jps
   ```

<br><br>

## 2. Gravando os Dados no Cluster HDFS

<br>

Nesta etapa, vamos copiar os arquivos do sistema local para o `HDFS` para que possam ser processados pelo `Mahout`.

<br>

#### 2.1 Criando Diretórios no HDFS

No terminal digitar os comandos:

```bash
hdfs dfs -mkdir /mahout/clustering
hdfs dfs -mkdir /mahout/clustering/data
```

#### 2.2 Lista os arquivos e diretórios no HDFS raiz.

```bash
hdfs dfs -ls /
```

#### 2.3 Copiando dados do sistema local para o HDFS

No diretório onde estão as pastas `ham` e `spam` ir ao terminal e digitar:

```bash
hdfs dfs -copyFromLocal news/* /mahout/clustering/data
```

#### 2.4 Verificar conteúdo

```bash
hdfs dfs -ls /mahout/clustering/data
hdfs dfs -cat /mahout/clustering/data/*
```

<br><br>


## 3. Pré-Processamento dos Dados com Apache Mahout

> **Importante**: O pré-processamento a seguir é específico para dados em formato de texto, como o conjunto de dados atual, que contém notícias armazenadas em arquivos `.txt`. A etapa de pré-processamento converte esses textos em um formato numérico que o Mahout pode utilizar para aplicar algoritmos de clusterização.

> O **objetivo do pré-processamento** é transformar as notícias em representações numéricas (vetores esparsos) usando o **TF-IDF** (Term Frequency-Inverse Document Frequency). Esses vetores ajudam o algoritmo de clusterização a identificar padrões e similaridades entre as notícias.

<br>

#### 3.1 Converter Dados para Formato de Sequence (obrigatório quando se trabalha com Mahout)

O primeiro passo no `Mahout` é transformar os dados de texto em uma **sequence file**, um formato binário que facilita o processamento distribuído no `Hadoop`.

```bash
mahout seqdirectory -i /mahout/clustering/data -o /mahout/clustering/kmeansseq
```

<br>

#### 3.2 Criar a Matriz Esparsa com Vetores TF-IDF

Com os dados em formato Sequence, o próximo passo é converter essas sequências para uma matriz esparsa de vetores **TF-IDF**. O TF-IDF mede a importância de cada termo em relação ao documento e ao conjunto total de documentos, permitindo que o Mahout utilize essas informações para agrupar notícias semelhantes.

```bash
mahout seq2sparse -i /mahout/clustering/kmeansseq -o /mahout/clustering/kmeanssparse
```

<br>

#### 3.2 Verificar Saída da Matriz Esparsa

Após a criação da matriz esparsa, verifique o diretório de saída para confirmar que os vetores **TF-IDF** foram gerados corretamente:

```bash
hdfs dfs -ls /mahout/clustering/kmeanssparse
```
<br>

A saída mostra os arquivos e diretórios gerados para representar os dados de texto em formato de vetores esparsos:

- **df-count**: Contagem de documentos por termo (usado para calcular **IDF**).
- **dictionary.file-0**: Mapeamento de termos para índices únicos.
- **frequency.file-0**: Frequência de termos no conjunto de dados.
- **tf-vectors**: Vetores de frequência dos termos (**TF**) por documento.
- **tfidf-vectors**: Vetores **TF-IDF**, que indicam a relevância de cada termo.
- **tokenized-documents**: Documentos tokenizados em palavras.
- **wordcount**: Contagem total de palavras. (`hadoop fs -text /mahout/output/sparseoutput/wordcount/*`)

<br>

Esses dados indicam que o pré-processamento está completo e pronto para uso em modelos de aprendizado de máquina.

<br><br>


## 4. Divisão dos Dados em Treino e Teste

<br>

Neste laboratório, não há divisão de dados em treino e teste porque estamos utilizando **aprendizado não supervisionado**.

O **objetivo** aqui é identificar grupos ou padrões nos dados sem rótulos pré-definidos.

No caso do **K-mean**s, o algoritmo é aplicado ao conjunto completo de dados para formar clusters, agrupando itens semelhantes. Como não estamos tentando prever uma categoria específica com base em rótulos conhecidos, não é necessário separar os dados para avaliação.

<br><br>

## 5. Treinando o Modelo com Dados no Cluster HDFS

<br>

Agora que os dados foram pré-processados e convertidos para o formato adequado, podemos aplicar o algoritmo de **K-means** usando o **Apache Mahout**. Este processo cria clusters de notícias semelhantes com base nos vetores **TF-IDF** dos termos em cada documento, agrupando-os em três categorias (conforme definido pelo parâmetro `-k 3`). Os centroids e os clusters resultantes serão armazenados no `HDFS` para análise posterior.

```bash
# Explicando os parâmetros
#	-i	diretório com arquivos de entrada
#	-c	diretório de destino para os centroids
#	-o	diretório de saída
#	-k	número de clusters
#	-ow	overwrite 
#	-x	número de iterações
#	-dm	medida de distância
mahout kmeans -i /mahout/clustering/kmeanssparse/tfidf-vectors/ -c /mahout/clustering/kmeanscentroids -cl -o /mahout/clustering/kmeansclusters -k 3 -ow -x 10 -dm org.apache.mahout.common.distance.CosineDistanceMeasure

# Visualiza os arquivos no HDFS
hdfs dfs -ls /mahout/clustering/kmeansclusters
```

<br><br>

## 6. Testando e Avaliando o Modelo

<br>

Após o treinamento do modelo `K-means`, **é possível exportar os clusters para um arquivo de texto e visualizar os resultados**. Isso permite entender o conteúdo de cada cluster e verificar os documentos atribuídos a cada grupo.

```bash
# Dump dos clusters para um arquivo texto
mahout clusterdump -i /mahout/clustering/kmeansclusters/clusters-1-final -o clusterdump.txt -p /mahout/clustering/kmeansclusters/clusteredPoints/ -d /mahout/clustering/kmeanssparse/dictionary.file-0 -dt sequencefile -n 20 -b 100

# Visualiza os clusters (salvo na pasta home da máquinalocal)
cat clusterdump.txt
```
<br>

Esse passo finaliza o laboratório e permite que você observe como os documentos foram agrupados em diferentes clusters com base em sua similaridade.

<br><br><br><br>

# Fim!